# Scyclone-PyTorch / your Dataset
[![Generic badge](https://img.shields.io/badge/GitHub-Scyclone--PyTorch-9cf.svg)][github]
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)][notebook]
[![Paper](http://img.shields.io/badge/paper-arxiv.2005.03334-B31B1B.svg)][paper]  

Reimplmentation of voice conversion system "Scyclone" with PyTorch  
Author: [tarepan](https://github.com/tarepan)

[github]:https://github.com/tarepan/Scyclone-PyTorch
[paper]:https://arxiv.org/abs/2005.03334
[notebook]:https://colab.research.google.com/github/tarepan/Scyclone-PyTorch/blob/main/Scyclone_PyTorch.ipynb

## Colab Check
Check
- Google Colaboratory runnning time
- GPU type
- Python version
- CUDA version

In [ ]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
!head -n 1 /proc/driver/nvidia/gpus/**/information
!cat /usr/local/cuda/version.txt

## Setup

Activate notebook intermittently for long session (RUN once **by hand**)
```javascript
const refresher = setInterval(()=>{document.querySelector("colab-connect-button").click();console.log("clicked for long session");}, 1000*60*10);
```

In [ ]:
# repository install
!pip uninstall scyclonepytorch -y -q
!pip install git+https://github.com/tarepan/Scyclone-PyTorch -q

## Argument Preparation

In [ ]:
from argparse import ArgumentParser

# args
arg_strs = ["--dir_exp", ".", "--max_epochs", "400000"]

# parse arguments for Scyclone-Pytorch
parser = ArgumentParser()
args_scpt = parseArgments(parser, arg_strs)

## Your Dataset Preparation
- Input datum: linear spectrogram (n_fft=254, from 16kHz waveform), described in original paper
- Input class: PyTorch-Lightning's `DataModule`

You should prepare PyTorch's [`torch.utils.data.dataset.Dataset`][dataset] which yield the spectrogram by yourself.  
Below code wrap two (non-parallel) datasets within DataModule.  

※ dataset preparation tips  
[`torchaudio.transforms.Spectrogram`][torch_audio_spec] transform wavefrom -> spectrogram.  
(Optional Resampling) + `Spectrogram(254)(waveform)` will help you.  
[npVCC2016] could be good sample dataset, good lack.  

[dataset]:https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset
[torch_audio_spec]:https://pytorch.org/audio/transforms.html#torchaudio.transforms.Spectrogram
[npVCC2016]:https://github.com/tarepan/npVCC2016/blob/main/npvcc2016/PyTorch/dataset/spectrogram.py

In [ ]:
# import your datasets and set them
dataset_A = <your dataset instance>
dataset_B = <your dataset instance>
n_data_val = <number of validation datum number in dataset>

# example
from npvcc2016.PyTorch.dataset.spectrogram import NpVCC2016_spec
dataset_A = NpVCC2016_spec(root=".", train=True, download=True, speakers=["SF1"])
dataset_B = NpVCC2016_spec(root=".", train=True, download=True, speakers=["TM3"])
n_data_val = 8

In [ ]:
class NonParallelSpecDataset(Dataset):
    """Combine two non-parallel datasets as single dataset which yield Tensor Tuple"""
    def __init__(self):
        self.datasetA = dataset_A
        self.datasetB = dataset_B

    def __getitem__(self, n: int):
        return (self.datasetA[n], self.datasetB[n])

    def __len__(self) -> int:
        return min(len(self.datasetA), len(self.datasetB))


class NonParallelSpecDataModule(LightningDataModule):
    def __init__(
        self,
        batch_size: int = 64,
        performance: DataLoaderPerformance = DataLoaderPerformance(4, True),
    ):
        super().__init__()
        self.batch_size = batch_size
        self._num_worker = performance.num_workers
        self._pin_memory = performance.pin_memory

    def prepare_data(self, *args, **kwargs) -> None:
        pass

    def setup(self, stage=None):
        if stage == "fit" or stage is None:
            dataset_full = NonParallelSpecDataset()
            
            mod = n_full % self.batch_size
            self.dataset_train, self.dataset_val = random_split(
                dataset_full, [len(dataset_full) - n_data_val, n_data_val]
            )
            self.batch_size_val = n_data_val
        if stage == "test" or stage is None:
            self.dataset_test = NonParallelSpecDataset()
            self.batch_size_test = self.batch_size

    def train_dataloader(self):
        return DataLoader(self.dataset_train, batch_size=self.batch_size, shuffle=True, num_workers=self._num_worker, pin_memory=self._pin_memory)

    def val_dataloader(self):
        return DataLoader(self.dataset_val, batch_size=self.batch_size_val, num_workers=self._num_worker, pin_memory=self._pin_memory)

    def test_dataloader(self):
        return DataLoader(self.dataset_test, batch_size=self.batch_size_test, num_workers=self._num_worker, pin_memory=self._pin_memory)

In [ ]:
loader_perf = DataLoaderPerformance(
    args_scpt.num_workers, not args_scpt.no_pin_memory
)

# `datamodule` is used for training
datamodule = NonParallelSpecDataModule(64, loader_perf)

## Train

In [ ]:
# Start TensorBoard
%load_ext tensorboard
%tensorboard

# Start Training
import pytorch_lightning as pl
from scyclonepytorch.Scyclone_main import train
from scyclonepytorch.args import parseArgments

# seed fixation
pl.seed_everything(1234)
# train
train(args_scpt, datamodule)